In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor
from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
import pandas as pd

In [ ]:
trainingData = pd.read_csv("train.csv")

X_train = trainingData.drop("SalePrice", axis = 1)
X_train
Y_train = trainingData.SalePrice

def encodeEntireData(X_train):
    cache = []
    for feature in X_train.columns.values:
        label_encoder =LabelEncoder()
        X_train[feature]= label_encoder.fit_transform(X_train[feature])
        cache.append(label_encoder)

    return cache

encodeEntireData(X_train)

X_train2, X_test, Y_train2, Y_test = train_test_split(X_train, Y_train, test_size=0.1, random_state=42)

In [ ]:
estimators = [#('rf1', RandomForestRegressor(n_estimators=10, random_state=42)),
              #('rf2', RandomForestRegressor(n_estimators=100, random_state=42)),
              #('rf3', RandomForestRegressor(n_estimators=19, random_state=42)),
              #('rf4', RandomForestRegressor(n_estimators=4, random_state=42)),
              ('rf5', RandomForestRegressor(n_estimators=300, max_leaf_nodes = 20, random_state=42)),
              ('rf6', RandomForestRegressor(n_estimators=300, max_leaf_nodes = 25, bootstrap=False, random_state=42)),
              ('rf7', RandomForestRegressor(n_estimators=100, max_leaf_nodes = 20, random_state=42)),
              #('xgboost1', XGBRegressor(learning_rate=0.1, max_depth=10, random_state=42)),
              #('xgboost2', XGBRegressor(learning_rate=0.1, max_depth=8, random_state=42)),
              #('xgboost3', XGBRegressor(n_estimators=100, learning_rate=0.01, max_depth=6, random_state=42)),
    
              ('xgboost4', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)),
    
              #('svr', LinearSVR(random_state=42)),
              #('svr', make_pipeline(StandardScaler(),LinearSVR(random_state=42))),
              ('nn1', MLPRegressor(hidden_layer_sizes = (1000,500,100,),random_state=42, max_iter=1000, early_stopping=True)),
              #('nn2', MLPRegressor(hidden_layer_sizes = (100,),random_state=42, max_iter=500, early_stopping=True)),
              #('nn3', MLPRegressor(hidden_layer_sizes = (10, 10,),random_state=42, max_iter=5000, early_stopping=True)),
              #('logReg', LogisticRegression()),
            # ('kmeans', make_pipeline(StandardScaler(),KMeans(random_state=42))),
              
             ]

clf = StackingRegressor(
     estimators=estimators#, final_estimator = LogisticRegression(random_state=42)
    #final_estimator=RandomForestRegressor(n_estimators=100, random_state=42)
 )

In [ ]:
clf.fit(X_train2, Y_train2)
print(clf.score(X_train2,Y_train2))
print(clf.score(X_test,Y_test))

In [ ]:
from sklearn.linear_model import LogisticRegression
logReg = LogisticRegression(random_state=0).fit(X_train2, Y_train2)
print(logReg.score(X_train2,Y_train2))
print(logReg.score(X_test,Y_test))

## with new features


In [ ]:
trainingData['YearsSinceRemodeled']  = trainingData['YrSold'] - trainingData['YearRemodAdd']
trainingData['YearsOld']  = trainingData['YrSold'] - trainingData['YearBuilt']
trainingData['YearsBeforeRemodel']  = trainingData['YearRemodAdd'] - trainingData['YearBuilt']
trainingData['YearsBeforeGarageBuild']  = trainingData['GarageYrBlt'] - trainingData['YearBuilt']
trainingData['TotalSF']  = trainingData['TotalBsmtSF'] + trainingData['1stFlrSF'] + trainingData['2ndFlrSF'] + trainingData['1stFlrSF']
trainingData['UnfinishedSF']  = (trainingData['LowQualFinSF'] + trainingData['BsmtUnfSF']) / trainingData['TotalSF']
trainingData['UnfinishedSFBasement']  = (trainingData['BsmtUnfSF']) / trainingData['TotalBsmtSF']
trainingData['FinishedSFBasement']  = 1 - (trainingData['BsmtUnfSF']) / trainingData['TotalBsmtSF']
trainingData['TotalOutsideSF']  = trainingData['WoodDeckSF'] + trainingData['OpenPorchSF'] + trainingData['EnclosedPorch'] + trainingData['3SsnPorch']+ trainingData['ScreenPorch'] + trainingData['PoolArea']
trainingData['PropertySF']  = trainingData['TotalSF'] + trainingData['TotalOutsideSF'] + trainingData['MasVnrArea'] + trainingData['LotFrontage']
trainingData['grassArea'] = trainingData['PropertySF']  - trainingData['LotArea'] 
trainingData['bathrooms'] = trainingData['BsmtFullBath']  + trainingData['BsmtHalfBath'] +trainingData['FullBath']  + trainingData['HalfBath'] 
trainingData['numberBedAndBath'] = trainingData['BedroomAbvGr']  + trainingData['FullBath'] 


In [ ]:
X_trainNew = trainingData.drop("SalePrice", axis = 1)
Y_trainNew = trainingData.SalePrice

encodeEntireData(X_trainNew)

X_train2New, X_testNew, Y_train2New, Y_testNew = train_test_split(X_trainNew, Y_trainNew, test_size=0.1, random_state=42)

In [ ]:
clf2 = StackingRegressor(
     estimators=estimators, final_estimator=RandomForestRegressor(n_estimators=10, random_state=42)
 )

clf.fit(X_train2New, Y_train2New)
print(clf.score(X_train2New,Y_train2New))
print(clf.score(X_testNew,Y_testNew))